# Analysis3 - extracting FiD encoder embedding 2

## CHECKING PARSER

In [3]:
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
import heapq
import pathlib
import shutil
from FiD.src.model import FiDT5
from src.model import FiDEncoderForSequenceClassification
import argparse

from pprint import pprint
from tqdm.auto import tqdm
from src.data import BinaryCustomDatasetShuffle

import json
import math
import os
import logging
import sys
import evaluate
from util import utils

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, SequentialSampler
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    set_seed,
    get_scheduler,
)
from util.arguments import ModelArguments, DataTrainingArguments, CustomTrainingArguments

from pprint import pprint
import numpy as np
import torch
from torch import nn
import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import T5PreTrainedModel
import copy
from transformers.modeling_outputs import SequenceClassifierOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from FiD.src.model import FiDT5
import FiD.src.data

In [4]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
# os.environ['CUDA_VISIBLE_DEVICES']='1'

In [5]:
parser = argparse.ArgumentParser(description='Extracting encoder embeddings for passage-level decoder (GPT2)')

parser.add_argument('--input_file_path', type=str, 
                    default = '/data/philhoon-relevance/decoder-classification/NQ-DEV-DPR/5-fold/1/ctx100id_split_train_1.json')
parser.add_argument('--output_file_path', type=str, 
                    default = '/data/philhoon-relevance/decoder-classification/NQ-DEV-DPR/5-fold/1/ctx100id_embedding_train_1.pickle')
parser.add_argument('--batch_size', type=int,
                    default=2)
parser.add_argument('--n_context', type=int,
                    default=5)
parser.add_argument('--max_seq_length', type=int,
                    default=200)
parser.add_argument('--model_name_or_path', type=str,
                    default = '/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large')

# args = parser.parse_args()
args = parser.parse_args(args = [])

In [4]:
# pprint(vars(args))

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [6]:
model_class = FiDT5

In [7]:
config = AutoConfig.from_pretrained(args.model_name_or_path)

In [8]:
model = model_class.from_pretrained(args.model_name_or_path)

In [9]:
# model.encoder => FiDT5.EncoderWrapper
# model.encoder.encoder => FiDT5.EncoderWrapper.encoder = T5 encoder Architecture w FiDT5 parameters
model_encoder = model.encoder.encoder

In [10]:
tokenizer = AutoTokenizer.from_pretrained('t5-base', return_dict=False)

/home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


## Get Input 

In [11]:
# print(args.input_file_path)
eval_data = args.input_file_path
# eval_fold = '/scratch/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/ctx100id_split_train_1.json'

In [12]:
print(eval_data)

/data/philhoon-relevance/decoder-classification/NQ-DEV-DPR/5-fold/1/ctx100id_split_train_1.json


In [13]:
eval_examples = FiD.src.data.load_data(
        eval_data,
    )

In [14]:
# len(eval_examples)

In [15]:
n_context = args.n_context

In [16]:
print(n_context)

5


In [17]:
eval_dataset = FiD.src.data.Dataset(
    eval_examples,  
    n_context = n_context
)

In [18]:
text_maxlength = args.max_seq_length

In [19]:
collator_function = FiD.src.data.Collator(text_maxlength, tokenizer, n_context)

In [20]:
eval_sampler = SequentialSampler(eval_dataset) 

In [21]:
per_gpu_batch_size = args.batch_size

In [22]:
print(per_gpu_batch_size)

2


In [23]:
eval_dataloader = DataLoader(
    eval_dataset, 
    sampler=eval_sampler, 
    batch_size=per_gpu_batch_size,
    num_workers=8,
    collate_fn=collator_function
)


In [49]:
model_encoder.to(device = device)
model_encoder.eval()
check = 0
result_dict = {}
n_passages = args.n_context
print(n_passages)

5


In [50]:
from tqdm.auto import tqdm

In [51]:
for index, _, _, passage_ids, passage_masks in tqdm(eval_dataloader):
    print(index)
    print(passage_ids.shape)
    print(passage_masks.shape)
    
    input_ids = passage_ids.view(passage_ids.size(0), -1)
    attention_mask = passage_masks.view(passage_masks.size(0), -1)
    
    print(input_ids.shape)
    print(attention_mask.shape)
    
    bsz, total_length = input_ids.shape
    passage_length = total_length // n_passages
    print(passage_length)
    
    input_ids = input_ids.view(bsz*n_passages, passage_length)
    print(input_ids.shape)
    
    attention_mask = attention_mask.view(bsz*n_passages, passage_length)
    print(attention_mask.shape)

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    
    outputs = model_encoder(input_ids, attention_mask)
    output_by_batch = outputs[0]
    output_by_batch = output_by_batch.view(bsz, n_passages, passage_length, -1)
    output_by = output_by_batch.detach().cpu()
    
    print(output_by.shape)
    for i in range(bsz):
        id_ = index[i].item()
        embedding = output_by[i,]
        result_dict[id_] = embedding
    
    check += 1
    if check == 100:
        break

/home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_

  0%|          | 0/3503 [00:00<?, ?it/s]

/home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_

tensor([0, 1])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([2, 3])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([4, 5])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([6, 7])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([8, 9])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([10, 11])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size

torch.Size([2, 5, 200, 1024])
tensor([92, 93])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([94, 95])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([96, 97])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([98, 99])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([100, 101])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([102, 103])
torch.Size([2,

torch.Size([2, 5, 200, 1024])
tensor([180, 181])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([182, 183])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([184, 185])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([186, 187])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([188, 189])
torch.Size([2, 5, 200])
torch.Size([2, 5, 200])
torch.Size([2, 1000])
torch.Size([2, 1000])
200
torch.Size([10, 200])
torch.Size([10, 200])
torch.Size([2, 5, 200, 1024])
tensor([190, 191])
torch.

In [38]:
result_dict[0].shape

torch.Size([5, 200, 1024])

In [39]:
result_dict[1].shape

torch.Size([5, 200, 1024])

In [40]:
result_dict[100].shape

torch.Size([5, 200, 1024])

In [41]:
result_dict[150].shape

torch.Size([5, 200, 1024])

In [45]:
print(args.output_file_path)

/data/philhoon-relevance/decoder-classification/NQ-DEV-DPR/5-fold/1/ctx100id_embedding_train_1.pickle


In [ ]:
with open(args.output_file_path, 'wb') as f:
    pickle.dump(new_instance, f)

In [46]:
result_dict[0]

tensor([[[-7.7715e-02,  8.5688e-02, -4.5739e-02,  ..., -2.1708e-01,
          -9.0992e-02, -9.1965e-03],
         [-1.2303e-01,  1.3432e-01, -1.2805e-04,  ..., -2.7478e-01,
          -2.6327e-02,  4.0313e-02],
         [ 1.5508e-01,  4.1932e-02,  1.4576e-01,  ...,  4.5006e-01,
          -2.3713e-01, -1.4170e-01],
         ...,
         [ 4.0225e-01, -1.3431e-01,  3.1150e-01,  ..., -1.3394e-01,
          -2.4641e-01,  3.5341e-01],
         [ 3.8914e-01, -1.2264e-01,  3.1169e-01,  ..., -1.4295e-01,
          -2.5223e-01,  3.7069e-01],
         [ 3.8478e-01, -1.0816e-01,  3.2307e-01,  ..., -1.5112e-01,
          -2.5433e-01,  3.6048e-01]],

        [[-1.5032e-01,  6.8224e-02, -2.6341e-02,  ..., -2.2828e-01,
          -7.3244e-02, -2.0166e-03],
         [-1.9287e-01,  1.3305e-01,  3.4472e-02,  ..., -2.6626e-01,
          -2.9993e-02,  5.5405e-02],
         [ 1.4436e-02,  4.2459e-02,  2.2709e-01,  ...,  3.7087e-01,
          -2.3722e-01, -1.4227e-01],
         ...,
         [ 4.4441e-01,  9

### Chekc result

In [10]:
import pickle

In [17]:
args.output_file_path

'/data/philhoon-relevance/decoder-classification/NQ-DEV-DPR/5-fold/1/ctx100id_embedding_train_1.pickle'

In [18]:
with open(args.output_file_path, 'rb') as f:
    result = pickle.load(f)

In [19]:
len(result)

100

In [20]:
result[0].shape

torch.Size([100, 200, 1024])

## Changing Names

In [1]:
# import pathlib 
# import re

In [2]:
# current_path = '/scratch/philhoon-relevance/decoder-classification/NQ-DEV-DPR/5-fold/1'

In [3]:
# path = pathlib.Path(current_path)

In [4]:
# files = path.glob('**/*')

# test_file = list(files)[0]

In [5]:
# test_file

In [6]:
# type(test_file.stem)

In [7]:
# filename = test_file.stem

In [8]:
# filename

In [9]:
# reg = 'ctx100id_embedding_train_1.picklectx100id_*'

In [10]:
# re.match(reg, filename)

In [11]:
# num = filename.split('_')[-1]

In [12]:
# 'ctx100id_embedding_train_1' in filename

# new_file_name = 'ctx100id_embedding_train_1' + f'_{num}'

In [13]:
# new_file_name

In [14]:
# previous = current_path + '/' + filename + '.pickle'

In [15]:
# previous

In [16]:
# new = current_path + '/' + new_file_name + '.pickle'

In [17]:
# print(new)

In [18]:
# os.rename(previous, new)

## Testing Files

In [21]:
import pickle

In [19]:
input_path = '/scratch/philhoon-relevance/decoder-classification/NQ-DEV-DPR/5-fold/1/ctx100id_embedding_train_1_1.pickle'

In [22]:
with open(input_path, 'rb') as f:
    data = pickle.load(f)

In [23]:
len(data)

100

In [28]:
data[7]['embedding'].shape

torch.Size([100, 200, 1024])